## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### Sesión Práctica 01 - Ejemplo mlflow

---

En esta libreta se realizan una serie de pasos básicos para verificar la operatoria a seguir con respecto al registro del entrenamiento y evaluación de modelos de aprendizaje automático utilizando la librería mlflow.

In [1]:
import mlflow
import numpy as np

from sklearn.datasets import load_iris
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
)

c:\Users\Maria Lasso\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Se realiza en primera instancia la lectura del conjunto de datos que será uno genérico.

In [2]:
db = load_iris()
features = db.data
target = db.target

Se ejecuta un método de clusterización haciendo uso de "autolog" de parte de la librería. Esta es una opción básica de registro que es de utilidad para los métodos de sklearn.

In [3]:
# Opción básica
mlflow.autolog()

# Clustering con DBSCAN.
dbscan = DBSCAN(eps=0.5, min_samples=5)

dbscan.fit_predict(features)

2026/02/28 10:47:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2026/02/28 10:47:53 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2026/02/28 10:47:53 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2026/02/28 10:47:54 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: [WinError 5] Acceso denegado: 'C:\\Users\\Maria%20Lasso'


array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1, -1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1, -1,  1,  1,  1,
        1,  1,  1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1,
        1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1, -1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [4]:
# Clustering con Kmeans.
kmeans = KMeans(n_clusters=3, random_state=42)

kmeans.fit_predict(features)

# davies_bouldin_score(features, kmeans.fit_predict(features))

2026/02/28 10:48:01 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: [WinError 5] Acceso denegado: 'C:\\Users\\Maria%20Lasso'


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2], dtype=int32)

Se muestran ejemplos con una denominación diferente para la experimentación a realizar a fin de dejar mayor registro.

In [ ]:
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# --- Configura el tracking local (opcional pero recomendado) ---
# Si quieres que guarde en una carpeta del proyecto:
# mlflow.set_tracking_uri("file:./mlruns")

exp_name = "Clustering-Ejemplos"

# Crear o reutilizar experimento
existing = mlflow.get_experiment_by_name(exp_name)
if existing is None:
    exp_id = mlflow.create_experiment(exp_name)
else:
    exp_id = existing.experiment_id

k = 2

with mlflow.start_run(experiment_id=exp_id, run_name=f"KMeans - K={k}"):
    modelo = KMeans(n_clusters=k, random_state=42, n_init="auto")
    modelo.fit(features)

    cluster_labels = modelo.labels_
    score = silhouette_score(features, cluster_labels)

    # params / metrics
    mlflow.log_param("value_of_k", k)
    mlflow.log_metric("silhouette_score", float(score))

    # input_example (mantener 2D)
    input_example = features[:1]
    mlflow.sklearn.log_model(modelo, "Clustering_Model", input_example=input_example)

In [ ]:
# Se puede hacer una experimentación nominal

exp_name = 'Clustering-Ejemplos'
exp_id = mlflow.create_experiment(name=exp_name)

with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=2"):
    modelo_clusters = KMeans(n_clusters=2)
    trained_model = modelo_clusters.fit(features)
    cluster_labels = trained_model.labels_
    score = silhouette_score(features, cluster_labels)
    #save parameter
    mlflow.log_param('value_of_k', 2)
    #save metric
    mlflow.log_metric('silhoutte_score', score)
    #save model
    input_example = np.array([features[0]])  # Example input
    mlflow.sklearn.log_model(trained_model, "Clustering_Model", input_example=input_example)
    #end current run
    mlflow.end_run()

PermissionError: [WinError 5] Acceso denegado: 'C:\\Users\\Maria%20Lasso'

In [7]:
with mlflow.start_run(experiment_id=exp_id, run_name="Kmeans - K=3"):
    modelo_clusters = KMeans(n_clusters=3)
    trained_model = modelo_clusters.fit(features)
    cluster_labels = trained_model.labels_
    score = silhouette_score(features, cluster_labels)
    score_2 = davies_bouldin_score(features, cluster_labels)
    #save parameter
    mlflow.log_param('value_of_k', 3)
    #save metric
    mlflow.log_metric('silhoutte_score', score)
    mlflow.log_metric('davies_bouldin_score', score_2)
    #save model
    input_example = np.array([features[0]])  # Example input
    mlflow.sklearn.log_model(trained_model, "Clustering_Model", input_example=input_example)
    #end current run
    mlflow.end_run()

NameError: name 'exp_id' is not defined

In [ ]:
with mlflow.start_run(experiment_id=exp_id, run_name="DBSCAN"):
    modelo_clusters = DBSCAN(eps=0.5, min_samples=5)
    trained_model = modelo_clusters.fit(features)
    cluster_labels = trained_model.labels_
    score=silhouette_score(features, cluster_labels)
    #save parameter
    mlflow.log_param('min_samples', 5)
    mlflow.log_param('eps', 0.5)
    #save metric
    mlflow.log_metric('silhoutte_score', score)
    #save model
    input_example = np.array([features[0]])  # Example input
    mlflow.sklearn.log_model(trained_model, "Clustering_Model", input_example=input_example)
    #end current run
    mlflow.end_run()

Este comando es el que permite visualizar la interfaz de mlflow en un navegador web. Se debe considerar ejecutarlo en la misma ubicación que contiene al directorio `mlruns`.

In [ ]:
!mlflow ui